# Chat Engine - ReAct Mode

ReAct is an agent based chat mode built on top of a query engine over your data.

For each chat interaction, the agent enter a ReAct loop:
* first decide whether to use the query engine tool and come up with appropriate input
* (optional) use the query engine tool and observe its output
* decide whether to repeat or give final response

This approach is flexible, since it can flexibility choose between querying the knowledge base or not.
However, the performance is also more dependent on the quality of the LLM. 
You might need to do more coercing to make sure it chooses to query the knowledge base at right times, instead of hallucinating an answer.

### Get started in 5 lines of code

Load data and build index

In [1]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from langchain.llms import OpenAI

service_context = ServiceContext.from_defaults(llm=OpenAI())
data = SimpleDirectoryReader(input_dir="../data/paul_graham/").load_data()
index = VectorStoreIndex.from_documents(data, service_context=service_context)

/Users/suo/miniconda3/envs/llama/lib/python3.9/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.7) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


Configure chat engine

In [ ]:
chat_engine = index.as_chat_engine(chat_mode="react", verbose=True)

Chat with your data

In [3]:
response = chat_engine.chat(
    "Use the tool to answer: what did Paul Graham do in the summer of 1995?"
)



> Entering new  chain...

Thought: Do I need to use a tool? Yes
Action: Query Engine Tool
Action Input: What did Paul Graham do in the summer of 1995?
Observation: 
In the summer of 1995, Paul Graham worked on building an application service provider (ASP) to allow people to edit code on a server through a browser, and then host the resulting applications for them. He recruited Dan Giffin, who had worked for Viaweb, and two undergrads who wanted summer jobs, and they got to work trying to build the software.
Thought: Do I need to use a tool? No
AI: That is correct, Paul Graham worked on building an application service provider (ASP) to allow people to edit code on a server through a browser, and then host the resulting applications for them.

> Finished chain.


In [4]:
print(response)

That is correct, Paul Graham worked on building an application service provider (ASP) to allow people to edit code on a server through a browser, and then host the resulting applications for them.


In [5]:
response = chat_engine.chat("What did I ask you?")



> Entering new  chain...

Thought: Do I need to use a tool? No
AI: You asked me what Paul Graham did in the summer of 1995.

> Finished chain.


In [6]:
print(response)

You asked me what Paul Graham did in the summer of 1995.


### Customize LLM

Use ChatGPT ("gpt-3.5-turbo")

In [7]:
from llama_index import ServiceContext
from langchain.chat_models import ChatOpenAI

service_context = ServiceContext.from_defaults(
    llm=ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo")
)

Configure chat engine

In [8]:
chat_engine = index.as_chat_engine(
    service_context=service_context, chat_mode="react", verbose=True
)

In [9]:
response = chat_engine.chat(
    "Use the tool to answer: what did Paul Graham do in the summer of 1995?"
)



> Entering new  chain...
{
    "action": "Query Engine Tool",
    "action_input": "What did Paul Graham do in the summer of 1995?"
}
Observation: Based on the given context information, it is not mentioned what Paul Graham did in the summer of 1995.
Thought:{
    "action": "Final Answer",
    "action_input": "Based on the available information, it is not mentioned what Paul Graham did in the summer of 1995."
}

> Finished chain.


In [10]:
print(response)

Based on the available information, it is not mentioned what Paul Graham did in the summer of 1995.


In [11]:
response = chat_engine.chat("What did I ask you before?")



> Entering new  chain...
{
    "action": "Query Engine Tool",
    "action_input": "What did Paul Graham do in the summer of 1995?"
}
Observation: Based on the given context information, it is not mentioned what Paul Graham did in the summer of 1995.
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


{
    "action": "Final Answer",
    "action_input": "Based on the available information, it is not mentioned what Paul Graham did in the summer of 1995."
}

> Finished chain.


In [12]:
print(response)

Based on the available information, it is not mentioned what Paul Graham did in the summer of 1995.


Reset chat engine

In [13]:
chat_engine.reset()

In [14]:
response = chat_engine.chat("What did I ask you before?")



> Entering new  chain...
{
    "action": "Query Engine Tool",
    "action_input": "What did I ask you before?"
}
Observation: Based on the given context information, it is not possible to determine what question you asked before.
Thought:{
    "action": "Query Engine Tool",
    "action_input": "What is the response to my last comment?"
}
Observation: The given context information does not provide any information about a previous comment or response. Therefore, it is not possible to determine the response to the last comment.
Thought:{
    "action": "Query Engine Tool",
    "action_input": "What is the response to my last comment?"
}
Observation: The given context information does not provide any information about a previous comment or response. Therefore, it is not possible to determine the response to the last comment.
Thought:{
    "action": "Query Engine Tool",
    "action_input": "What is the response to my last comment?"
}
Observation: The given context information does not prov

In [15]:
print(response)

Agent stopped due to iteration limit or time limit.


### Customize Personality (via System Prompt)

In [ ]:
from llama_index.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

chat_engine = index.as_chat_engine(
    chat_mode="react",
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
    verbose=True, 
)

In [18]:
response = chat_engine.chat("Hi, tell me a tragic story")



> Entering new  chain...

Thought: Do I need to use a tool? No
AI: 'Tis a tale of woe, of a star-crossed lovers Romeo and Juliet. They be of two feuding families, and yet their hearts were true. Fate had other plans, and in the end, only sadness and despair remained.

> Finished chain.
